In [1]:
#%%
import pandas as pd
import numpy as np
import joblib
from pyLDAvis import display
import matplotlib.pyplot as plt
pd.set_option('display.max_columns',None)

/Users/beomso0/.local/share/virtualenvs/deploy_st-PF8VvhC-/lib/python3.9/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [2]:
test_df = joblib.load('process_test.pkl')
midcat_ratio = joblib.load('midcat_ratio.pkl')
goal_df = joblib.load('0216_0357_skill2pgm_formodel_ver1.pkl')

In [3]:
test_df

,showhost_input,weather_input,weekday_input,brand_input,midcat_input,price_input,expression_input,year_input,month_input,time_input,duration_input,prd_num_input,live_input,holiday_input
0,"[왕영은, 하은지]",흐림,수,"[에이앤디, LG(엘지), 자연식품]","[여성캐쥬얼, 생활가전, 가공식품]","[55, 1697, 33]","[기간한정, 마지막(생방송)]",2022,3,19,120,3,True,False


In [11]:
t = test_df.copy()
int_cols = [
  'year_input',
  'month_input',
  'time_input',
  'duration_input',
  'prd_num_input'
]
t[int_cols] = t[int_cols].astype(int)

In [10]:
test_df.month_input.values[0]

3

In [8]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   showhost_input    1 non-null      object
 1   weather_input     1 non-null      object
 2   weekday_input     1 non-null      object
 3   brand_input       1 non-null      object
 4   midcat_input      1 non-null      object
 5   price_input       1 non-null      object
 6   expression_input  1 non-null      object
 7   year_input        1 non-null      object
 8   month_input       1 non-null      object
 9   time_input        1 non-null      object
 10  duration_input    1 non-null      object
 11  prd_num_input     1 non-null      object
 12  live_input        1 non-null      object
 13  holiday_input     1 non-null      object
dtypes: object(14)
memory usage: 240.0+ bytes


In [13]:
# 전처리 함수s (sub)
def encode(data, col, max_val):
    data[col + '_sin'] = np.sin(2 * np.pi * data[col]/max_val)
    data[col + '_cos'] = np.cos(2 * np.pi * data[col]/max_val)
    return data

In [14]:
# 전처리 함수 만들기 (main)

def process_input(df):
  # 자료형 변환
  int_cols = [
    'year_input',
    'month_input',
    'time_input',
    'duration_input',
    'prd_num_input'
  ]
  df[int_cols] = df[int_cols].astype(int)

  # col name change
  df = df.rename(columns={
    'year_input':'year',
    'month_input':'month',
    'showhost_input': '쇼호스트명',
    'weather_input': '날씨',
    'weekday_input': 'weekday',
    'duration_input': 'duration',
    'prd_num_input': 'prod_num',
    'live_input': '상품구분',
    'holiday_input':'주말/공휴일'
  })

  # 쇼호스트명 sorting
  df['쇼호스트명'] = df['쇼호스트명'].apply(lambda x: sorted(x))

  # 상품구분 변환
  df['상품구분'] = df['상품구분'].apply(lambda x: 'LIVE' if x else 'DATA')
  # 주말/공휴일 변환
  df['주말/공휴일'] = df['주말/공휴일'].apply(lambda x: 1 if x else 0)

  #year, month, hour
  df['year'] = df['year']-2020
  df['month' + '_sin'] = np.sin(2 * np.pi * df['month']/12)
  df['month' + '_cos'] = np.cos(2 * np.pi * df['month']/12)
  df['hour' + '_sin'] = np.sin(2 * np.pi * df['time_input']/24)
  df['hour' + '_cos'] = np.cos(2 * np.pi * df['time_input']/24)

  #판매가
  df['판매가평균'] = df['price_input'].apply(lambda x: np.mean(x)*1000)
  df['price_max_all'] = df['price_input'].apply(lambda x: max(x)*1000)
  df['price_min_all'] = df['price_input'].apply(lambda x: min(x)*1000)
  
  #한정표현 num
  df['한정표현_num'] = df['expression_input'].apply(lambda x: len(x))

  # sh_num
  df['sh_num'] = df['쇼호스트명'].apply(lambda x: len(x))


  return df
  

In [15]:
t = test_df.copy()
out = process_input(t)

In [16]:
out.head()

,쇼호스트명,날씨,weekday,brand_input,midcat_input,price_input,expression_input,year,month,time_input,duration,prod_num,상품구분,주말/공휴일,month_sin,month_cos,hour_sin,hour_cos,판매가평균,price_max_all,price_min_all,한정표현_num,sh_num
0,"[왕영은, 하은지]",흐림,수,"[에이앤디, LG(엘지), 자연식품]","[여성캐쥬얼, 생활가전, 가공식품]","[55, 1697, 33]","[기간한정, 마지막(생방송)]",2,3,19,120,3,LIVE,0,1.0,6.123234e-17,-0.965926,0.258819,595000.0,1697000,33000,2,2


In [ ]:
goal_df.head(1)

,인입콜,year,쇼호스트명,상품구분,날씨,weekday,duration,판매가평균,price_max_all,price_min_all,prod_num,한정표현_num,주말/공휴일,sh_num,month_sin,month_cos,hour_sin,hour_cos,시차변수,week,price_max_top3,price_min_top3,num1,num2,num3,brand1,brand2,brand3,price1,price2,price3,score1,score2,score3
0,33,0.0,"[엄지혜, 이경은]",DATA,구름 많음,수,40,99000.0,99000,99000,1,0,0,2.0,0.5,0.866025,1.0,6.123234e-17,NaN,1,99000,99000,3,3,3,소담,소담,소담,99000,99000,99000,0.045988,0.045988,0.045988


In [ ]:
goal_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34051 entries, 0 to 34050
Data columns (total 34 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   인입콜             34051 non-null  int64  
 1   year            34051 non-null  float64
 2   쇼호스트명           30949 non-null  object 
 3   상품구분            34051 non-null  object 
 4   날씨              34051 non-null  object 
 5   weekday         34051 non-null  object 
 6   duration        34051 non-null  int64  
 7   판매가평균           34051 non-null  float64
 8   price_max_all   34051 non-null  int64  
 9   price_min_all   34051 non-null  int64  
 10  prod_num        34051 non-null  int64  
 11  한정표현_num        34051 non-null  int64  
 12  주말/공휴일          34051 non-null  int64  
 13  sh_num          30949 non-null  float64
 14  month_sin       34051 non-null  float64
 15  month_cos       34051 non-null  float64
 16  hour_sin        34051 non-null  float64
 17  hour_cos        34051 non-null 